# Forest Fire Detection with transfer learning using VGG19

In [41]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers
from keras.models import load_model
from keras.applications import VGG19
from keras import layers
from keras import Model
import matplotlib.pyplot as plt
%matplotlib inline

In [42]:
train_dir = "data/Training"
test_dir = "data/Test"

VGG_SIZE = 224

In [43]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                rotation_range=40,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                validation_split=0.25)

test_datagen = ImageDataGenerator(rescale=1./255)

In [44]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                target_size=(VGG_SIZE, VGG_SIZE), 
                                                color_mode='rgb',
                                                batch_size=20,
                                                subset='training',
                                                class_mode='binary')

val_generator = train_datagen.flow_from_directory(train_dir,
                                                target_size=(VGG_SIZE, VGG_SIZE), 
                                                color_mode='rgb',
                                                batch_size=20,
                                                subset='validation',
                                                class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                target_size=(VGG_SIZE, VGG_SIZE),
                                                color_mode='rgb',
                                                batch_size=20,
                                                class_mode='binary')

Found 29547 images belonging to 2 classes.
Found 9848 images belonging to 2 classes.
Found 8617 images belonging to 2 classes.


In [45]:
transfer_model = VGG19(include_top=True,
                        weights="imagenet",
                        classes=1000)
input = transfer_model.inputs

In [46]:
x = transfer_model.get_layer("block5_pool").output
x = layers.Flatten(name='flatten')(x)
x = layers.Dense(2048, activation='relu')(x)
x = layers.Dense(2048, activation="relu")(x)
output_layer = layers.Dense(1, activation='sigmoid')(x)

classifier = Model(input, output_layer)

for layer in classifier.layers[:-3]:
    layer.trainable = False

classifier.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [47]:
classifier.compile(loss="binary_crossentropy",
                optimizer=optimizers.Adam(lr=0.001),
                metrics=["accuracy"])

In [49]:
callbacks = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath='models/vgg19_model.h5', save_best_only=True),
]

history = classifier.fit(train_generator, 
                        epochs=5, 
                        batch_size=40,
                        validation_data=val_generator,
                        callbacks=callbacks)

Epoch 1/20
1478/1478 [==============================] - 6328s 4s/step - loss: 0.1088 - accuracy: 0.9602 - val_loss: 0.0356 - val_accuracy: 0.9857
Epoch 2/20
1478/1478 [==============================] - 6332s 4s/step - loss: 0.0676 - accuracy: 0.9737 - val_loss: 0.1285 - val_accuracy: 0.9466
Epoch 3/20
1478/1478 [==============================] - 6268s 4s/step - loss: 0.0509 - accuracy: 0.9811 - val_loss: 0.3403 - val_accuracy: 0.9238
Epoch 4/20
1478/1478 [==============================] - 6278s 4s/step - loss: 0.0434 - accuracy: 0.9839 - val_loss: 0.1088 - val_accuracy: 0.9584
Epoch 5/20
1478/1478 [==============================] - 6261s 4s/step - loss: 0.0413 - accuracy: 0.9849 - val_loss: 0.1051 - val_accuracy: 0.9617
Epoch 6/20
  88/1478 [>.............................] - ETA: 1:14:06 - loss: 0.0362 - accuracy: 0.9869

KeyboardInterrupt: 

In [51]:
classifier.save("models/vgg19_model.h5")

In [55]:
result = classifier.evaluate(test_generator)
print(f"Test loss: {result[0]}, Test accuracy: {result[1]}")

493/493 [==============================] - 1468s 3s/step - loss: 0.1799 - accuracy: 0.9301
Test loss: 0.17991960048675537, Test accuracy: 0.930138111114502
